In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import re
import os
from scipy import stats
from scripts.utils.my_fdr import fdr_threshold
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt


In [12]:
df_new_pheno = pd.read_csv('data/organ_pca.csv', index_col="eid")


df_disease = pd.read_csv('data/disease_with_pn.csv', index_col=0)
df_covariate = pd.read_csv("data//covariates.tsv", index_col=0, sep="\t")
df_covariate.index.name = "eid"

In [13]:
diseases = list(df_disease.columns)
phenotypes = list(df_new_pheno.columns)
phenotype_name = dict(zip(phenotypes, df_new_pheno.columns.str.removeprefix("y_res_")))



phenotype_unit = list(df_new_pheno.columns)

In [14]:
# For each disease of interest, perform logistic regression
#
# disease ~ Sex + Age + Weight + Height + Imaging_Phenotype
#
n_disease = len(diseases)
n_pheno = len(phenotypes)
df_beta = pd.DataFrame(index=diseases, columns=phenotypes, dtype=float)
df_conf = df_beta.copy()
df_p_val = df_beta.copy()
df_n = {}

In [15]:
odd_df = pd.DataFrame()
for pheno in phenotypes:
    for disease in diseases:

        # Dependent variables
        y = df_disease[disease]
        y = y.loc[((y == 0) | (y == 1))]
        y = y.replace(-1, 1)
        # Independent variables
        # X = pd.concat([df_info[['Sex', 'Age']], df[[pheno]]], axis=1)
        X = pd.merge(df_covariate[['isMale', 'age', 'weight', 'height']], df_new_pheno[[pheno]], on="eid", how="inner")

        # Standard deviation of the phenotype for normalisation
        pheno_std = df_new_pheno[pheno].std()

        # Remove NaN values before regression
        valid_idx = pd.merge(X, y, on="eid", how="inner").dropna().index
        df_n[pheno] = len(valid_idx)
        
        try:
            # Logistic regression
            model = sm.Logit(y.loc[valid_idx], X.loc[valid_idx]).fit(disp=False)
            alpha = 0.05
            t = stats.t.isf(alpha / 2, model.df_resid)

            df_beta.loc[disease, pheno] = model.params[pheno] * pheno_std
            df_conf.loc[disease, pheno] = t * model.bse[pheno] * pheno_std
            df_p_val.loc[disease, pheno] = model.pvalues[pheno]

            odds = np.exp(df_beta.loc[disease, pheno])
            odds_lower = np.exp(df_beta.loc[disease, pheno] - df_conf.loc[disease, pheno])
            odds_upper = np.exp(df_beta.loc[disease, pheno] + df_conf.loc[disease, pheno])
            # print('{0}, per {1:.1f} {2}: Odds = {3:.2f} [{4:.2f}, {5:.2f}], p = {6:.3f}'.format(
            #     disease, pheno_std, pheno,
            #     odds, odds_lower, odds_upper, df_p_val.loc[disease, pheno]))
            temp = pd.DataFrame({"pheno": [pheno], "disease": [disease], "odds": [odds], "odds_lower": [odds_lower], "odds_upper": [odds_upper], "p": [df_p_val.loc[disease, pheno]]})
            odd_df = pd.concat([odd_df, temp])
        except:
            pass
odd_df.to_csv("data/pca_disease_odd_after.csv", index=False)

c:\Users\liumeng\miniconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\liumeng\miniconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
c:\Users\liumeng\miniconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\liumeng\miniconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
c:\Users\liumeng\miniconda3\Lib\site-packages\statsmodels\discrete\discrete_model.py:2441: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q * linpred)))
c:\Users\liumeng\miniconda3\Lib\site-packages\statsmodels\base\model.py:60